<a href="https://colab.research.google.com/github/graykar/Deep_Learning_Model/blob/main/CNN_Fashion_Mnist_Guru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 6.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=5fd4208b5035114c3b563ed1d92ae65976cc6dc55da54bf2c350734ee61e81d0
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=5d90c82fb3e3ab96434d41c4f12eb29204c48f2471995d8ae2c723583f560d51
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images ,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images.shape , train_labels.shape

((60000, 28, 28), (60000,))

In [7]:
train_labels

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [8]:
train_images.max() , test_images.max()

(255, 255)

In [9]:
train_images=train_images/255.0
test_images=test_images/255.0

In [10]:
train_images.max() , test_images.max()

(1.0, 1.0)

In [11]:
train_images.shape , test_images.shape

((60000, 28, 28), (10000, 28, 28))

In [12]:
len(train_images)

60000

In [13]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [14]:
train_images.shape 

(60000, 28, 28, 1)

In [15]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),  # Filter Size to mention like Vertical or Horizontal
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=16, max_value=64, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [16]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [17]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=10,directory='output',project_name="Mnist Fashion")

In [18]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 10 Complete [00h 00m 15s]
val_accuracy: 0.10083333402872086

Best val_accuracy So Far: 0.9120000004768372
Total elapsed time: 00h 03m 26s
INFO:tensorflow:Oracle triggered exit


In [20]:
model=tuner_search.get_best_models(num_models=1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        38432     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                991296    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 1,035,018
Trainable params: 1,035,018
Non-trainable params: 0
______________________________________________

In [22]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1646 - accuracy: 0.9381 - val_loss: 0.2624 - val_accuracy: 0.9110
Epoch 5/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1181 - accuracy: 0.9567 - val_loss: 0.2687 - val_accuracy: 0.9143
Epoch 6/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0899 - accuracy: 0.9663 - val_loss: 0.2880 - val_accuracy: 0.9175
Epoch 7/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0638 - accuracy: 0.9756 - val_loss: 0.3329 - val_accuracy: 0.9128
Epoch 8/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0512 - accuracy: 0.9809 - val_loss: 0.3789 - val_accuracy: 0.9097
Epoch 9/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0391 - accuracy: 0.9859 - val_loss: 0.3698 - val_accuracy: 0.9095
Epoch 10/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0320 - accuracy: 0.9881 - val_loss: 0.4266 - val_accuracy